# How to use OpenAI function calling with LLMs
Function calling extends the power capabilities of LLMs. It allolws you to format
the output of an LLM response into a JSON object, which then can be fed down stream
to an actual function as a argument to process the response.

OpenAI documention states the basic steps involved in function calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

<img src="./images/gpt_function_calling.png">

Let's first demonstrate how we use this feature. Let's first specify a function and use the API to generate function arguments.

## How to generate function arguments


In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Function calling for Anyscale Endpoints is coming soon. Not yet ready as of writing this notebook!

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-0613; base=https://api.openai.com/v1


Define some utitilty functions

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
def add_prime_number_commpletion(clnt: object, model: str, user_content:str) -> object:
    chat_completion = clnt.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": user_content}],
    functions = [{
            "name": "add_prime_numbers",
            "description": "Add a list of 25 integer prime numbers between 2 and 100",
            "parameters": {
                "type": "object",
                "properties": {
                    "prime_numbers": {
                        "type": "array",
                        "items": {
                            "type": "integer",
                            "description": "A list of 25 prime numbers"
                        },
                        "description": "List of of 25 prime numbers to be added"
                    }
                },
                "required": ["add_prime_numbers"]
            }
        }],
        function_call={"name": "add_prime_numbers"}
    )
    response = chat_completion.choices[0].message
    return response

In [5]:
# Define a function which will be fed a dictionary with a key
# holding a list of 25 randomly generated prime numbers between
# 2 and 100
from typing import List, Dict
def add_prime_numbers(p_numbers: Dict[str, List[int]]) -> int:
    return sum(p_numbers["prime_numbers"])

Define the functions argument and function specification as part of the message


In [6]:
user_content = "Add 25 randomly generated prime numbers between 2 and 100"

In [7]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response = add_prime_number_commpletion(client, MODEL, user_content)
print(response)

Using Endpoints: https://api.openai.com/v1 ...

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "prime_numbers": [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]\n}', name='add_prime_numbers'), tool_calls=None)


#### Example 1: Extract the generated arguments
Let's process generated arguments to compute the sum of random
prime numbers between 2 and 100. These is a list fed into a function to compute its sum.

The idea is here is to nudge the LLM to generate JSON object, which can be easily converted into a Python dictionary as an  argument to a function downstream to compute the sum.

We can convert the resonse into a dictionary

In [8]:
import json
json_arguments = response.dict()
json_arguments

{'content': None,
 'role': 'assistant',
 'function_call': {'arguments': '{\n  "prime_numbers": [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]\n}',
  'name': 'add_prime_numbers'},
 'tool_calls': None}

In [9]:
# Extract specific items from the dict
func_name = json_arguments['function_call']['name']
print(f"Function name: {func_name}")

Function name: add_prime_numbers


In [10]:
funcs = json_arguments['function_call']['arguments']
funcs_args = json.loads(funcs)
print(f"Arguments: {funcs_args}")

Arguments: {'prime_numbers': [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]}


### Call the function from within our notebook.
This is our downstream function being invoked with the extract arguments
from the JSON response generated by the LLM.

In [11]:
sum_of_prime = add_prime_numbers(funcs_args)
sum_of_prime

1060

### Send the function response to LLM 
Embedded the function in a message and resend it to the LLM for execution.
This returns the sum of the randomly generated list of prime numbers.

In [12]:
user_content = "Sum the random prime numbers in the given list"

In [13]:
def llm_add_prime_number_commpletion(clnt: object, model: str, 
                                     user_content:str,
                                     func_name: str,
                                     llm_response: object) -> object:
    chat_completion = clnt.chat.completions.create(
    model=model,
    messages=[
        { "role": "user", "content": user_content},
          llm_response,              # the first message returned from LLM
           {
                "role": "function",  # role is function call
                "name": func_name,   # name of the function
                "content": "Add primes numbers in a list",  # content discription
                },
            ],
        )
    return chat_completion
                                     

In [14]:
print(f"Using Endpoints: {openai.api_base} ...\n")
second_response = llm_add_prime_number_commpletion(client, MODEL, user_content, func_name, response)
print(second_response)

Using Endpoints: https://api.openai.com/v1 ...

ChatCompletion(id='chatcmpl-8XCaYy9rR8jGflZKPn77SEojIa0IQ', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The sum of the prime numbers in the list is 1060.', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1702923906, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=120, total_tokens=134))


In [15]:
# Extract the content from the returned response
print(second_response.choices[0].message.content)

The sum of the prime numbers in the list is 1060.


#### Example 2: Extract the generated arguments

Let's process generated arguments to plot a map of cities where we hold 
Ray meetups. These generated satellite coordinates, generated as a JSON object
by the LLM, are fed into a function to generate an HTML file and map markers of
each city.

The idea is here is to nudge the LLM to generate JSON object, which can be easily converted into a Python dictionary as an argument to a function downstream to compute the sum.

In [16]:
def generate_city_map_commpletion(clnt: object, model: str, user_content:str) -> object:
    chat_completion = clnt.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": user_content}],
    functions = [{
            "name": "generate_ray_meetup_map",
            "description": "Generate HTML map for global cities where Ray meetups are hosted",
            "parameters": {
                "type": "object",
                "properties": {
                    "coordinates": {
                        "type": "array",
                        "items": {
                            "location": {
                                "type": "string",
                                "description": "The city name e.g., San Francisco, CA",
                            },
                            "latitude": {
                                "type": "string",
                                "description": "Latitude satelite coordinates",
                            },
                            "longitude": {
                                "type": "string",
                                "description": "Longitude satelite coordinates",
                            },
                        },

                    },
                },
            },
            "required": ["coordinates"]
        }],
        function_call={"name": "generate_ray_meetup_map"}
    )
    response = chat_completion.choices[0].message
    return response

In [24]:
user_content = """Generate satelite coordinates, including longitude,latitude and city, for Ray meetup hosted for each city: 
San Franciscto, New York, and London.""" 

In [25]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response = generate_city_map_commpletion(client, MODEL, user_content)
print(response)

Using Endpoints: https://api.openai.com/v1 ...

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"coordinates": [\n    {\n        "latitude": 37.7749,\n        "longitude": -122.4194,\n        "city": "San Francisco"\n    },\n    {\n        "latitude": 40.7128,\n        "longitude": -74.0060,\n        "city": "New York"\n    },\n    {\n        "latitude": 51.5074,\n        "longitude": -0.1278,\n        "city": "London"\n    }\n]\n}', name='generate_ray_meetup_map'), tool_calls=None)


In [26]:
json_arguments = response.dict()
json_arguments

{'content': None,
 'role': 'assistant',
 'function_call': {'arguments': '{\n"coordinates": [\n    {\n        "latitude": 37.7749,\n        "longitude": -122.4194,\n        "city": "San Francisco"\n    },\n    {\n        "latitude": 40.7128,\n        "longitude": -74.0060,\n        "city": "New York"\n    },\n    {\n        "latitude": 51.5074,\n        "longitude": -0.1278,\n        "city": "London"\n    }\n]\n}',
  'name': 'generate_ray_meetup_map'},
 'tool_calls': None}

In [34]:
# Extract specific items from the dict
func_name = json_arguments['function_call']['name']
print(f"Function name: {func_name}")

Function name: generate_ray_meetup_map


In [28]:
funcs = json_arguments['function_call']['arguments']
funcs_args = json.loads(funcs)
print(f"Arguments: {funcs_args}")

Arguments: {'coordinates': [{'latitude': 37.7749, 'longitude': -122.4194, 'city': 'San Francisco'}, {'latitude': 40.7128, 'longitude': -74.006, 'city': 'New York'}, {'latitude': 51.5074, 'longitude': -0.1278, 'city': 'London'}]}


In [29]:
import folium

def create_map(path: str, coordinates: Dict[str, List[object]]) -> None:
    # Create a base map
    m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
    coordinates_list = coordinates["coordinates"]
    for coordindates in coordinates_list:
        # Adding markers for each city
        folium.Marker([coordindates["latitude"], coordindates["longitude"]], popup=coordindates["city"]).add_to(m)
    # Display the map
    m.save(path)

#### Invoke the function from within our notebook.
This is our downstream function being invoked with the extract arguments
from the JSON response generated by the LLM.

In [32]:
from IPython.display import IFrame

# Assuming the HTML file is named 'example.html' and located in the same directory as the Jupyter Notebook
html_file_path = './world_map_nb_func_with_cities.html'
create_map(html_file_path, funcs_args)

# Display the HTML file in the Jupyter Notebook
IFrame(src=html_file_path, width=700, height=400)

### Send the function response to LLM 
Embed the function role in a message and resend it to the LLM for execution.
This will generate the HTML that can be saved in a file by a different name so that
we can distinguish between the two calls: one explicitly from the notebook and
the other from LLM: both ought to generate identical map with cooridnates.

In [35]:
def llm_generate_city_map_commpletion(clnt: object, model: str, user_content:str, 
                                      func_name: str, llm_response: object) -> object:
    chat_completion = clnt.chat.completions.create(
        model=model,
        messages=[ { "role": "user", "content": user_content},
                llm_response,              # the first message returned from LLM
                {
                    "role": "function",  # role is function call
                    "name": func_name,   # name of the function
                    "content": "Generate an HTML for the global coordinates provided for the Ray Meetup",  # content discription
                },
            ],
        )
    return chat_completion

In [36]:
print(f"Using Endpoints: {openai.api_base} ...\n")
second_response = llm_generate_city_map_commpletion(client, MODEL, user_content, func_name, response)
print(second_response)

Using Endpoints: https://api.openai.com/v1 ...

ChatCompletion(id='chatcmpl-8XCnFgYSyas9B4OdOLBvzW4VkPHO5', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='<html>\n<head>\n    <title>Ray Meetup Locations</title>\n    <style>\n        #map {\n            height: 500px;\n            width: 100%;\n        }\n    </style>\n</head>\n<body>\n    <h3>Locations of Ray Meetups</h3>\n    <div id="map"></div>\n    <script>\n        function initMap() {\n            var locations = [\n                {lat: 37.7749, lng: -122.4194, city: \'San Francisco\'},\n                {lat: 40.7128, lng: -74.0060, city: \'New York\'},\n                {lat: 51.5074, lng: -0.1278, city: \'London\'}\n            ];\n\n            var map = new google.maps.Map(document.getElementById(\'map\'), {\n                zoom: 2,\n                center: {lat:45 , lng: 0}\n            });\n\n            var markers = locations.map(function(location) {\n                var marker = ne

In [42]:
# Extract the content from the returned response
llm_html_content = second_response.choices[0].message.content
print(llm_html_content)

<html>
<head>
    <title>Ray Meetup Locations</title>
    <style>
        #map {
            height: 500px;
            width: 100%;
        }
    </style>
</head>
<body>
    <h3>Locations of Ray Meetups</h3>
    <div id="map"></div>
    <script>
        function initMap() {
            var locations = [
                {lat: 37.7749, lng: -122.4194, city: 'San Francisco'},
                {lat: 40.7128, lng: -74.0060, city: 'New York'},
                {lat: 51.5074, lng: -0.1278, city: 'London'}
            ];

            var map = new google.maps.Map(document.getElementById('map'), {
                zoom: 2,
                center: {lat:45 , lng: 0}
            });

            var markers = locations.map(function(location) {
                var marker = new google.maps.Marker({
                    position: location,
                    map: map
                });
                var infowindow = new google.maps.InfoWindow({
                    content: location.city
            

In [43]:
file_path = './world_map_llm_func_with_cities.html'
with open(file_path, 'w') as file:
        file.write(llm_html_content)

In [44]:
# Display the HTML file in the Jupyter Notebook
IFrame(src=html_file_path, width=700, height=400)

As you see, both methods of python function calling generated the
same map. 